[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/CamiloVga/Curso-IA-Aplicada/blob/main/Semana%2011_Arquitectura%20Transformers/Script_Clase_22_Transformers.ipynb)

# 🤖 Inteligencia Artificial Aplicada para la Economía
## Universidad de los Andes

### 👨‍🏫 Profesores
- **Profesor Magistral:** [Camilo Vega Barbosa](https://www.linkedin.com/in/camilovegabarbosa/)
- **Asistente de Docencia:** [Sergio Julian Zona Moreno](https://www.linkedin.com/in/sergiozonamoreno/)

### 📚 Implementación de Transformers con Llama-2
Este script implementa un chatbot basado en la arquitectura Transformer utilizando el modelo Llama-2-7b de Meta:

1. **Configuración y Optimización del Modelo Transformer 🚀**
   * Carga optimizada de Llama-2-7b con quantización de 8-bit
   * Configuración de precisión de 16 bits para eficiencia en GPU
   * Distribución automática del modelo para optimizar recursos
   * Preparación del modelo para ejecución en GPUs de Colab

2. **Generación de Texto con Arquitectura Transformer 🧠**
   * Tokenización y procesamiento de entradas de texto
   * Configuración de parámetros de sampling probabilístico
   * Generación controlada mediante temperature y top_p
   * Decodificación eficiente de tokens a texto natural

3. **Interfaz Conversacional con Gradio 💬**
   * Implementación de interfaz de usuario intuitiva
   * Manejo de contexto y formato específico para Llama-2
   * Ejemplos predefinidos sobre conceptos de Transformers
   * Diseño responsivo para facilitar la experimentación


In [ ]:
# Instalar bibliotecas necesarias
!pip install transformers torch accelerate bitsandbytes gradio -q

# Importar bibliotecas
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch

# Cargar tokenizador
tokenizer = AutoTokenizer.from_pretrained("meta-llama/Llama-2-7b-hf")

# Cargar modelo con optimizaciones para T4
model = AutoModelForCausalLM.from_pretrained(
    "meta-llama/Llama-2-7b-hf",
    device_map="auto",           # Distribuye el modelo automáticamente
    torch_dtype=torch.float16,   # Usa precisión de 16 bits
    load_in_8bit=True            # Carga el modelo con cuantización de 8 bits
)

# Generar texto
prompt = "Explica de manera simple qué es un transformer en inteligencia artificial:"
inputs = tokenizer(prompt, return_tensors="pt").to("cuda")  # Mover a GPU
outputs = model.generate(
    inputs.input_ids,
    max_new_tokens=200,
    do_sample=True,
    temperature=0.7
)
response = tokenizer.decode(outputs[0], skip_special_tokens=True)
print(response)

In [ ]:

# Importar bibliotecas
import gradio as gr


# Historial del chat para mantener la conversación
conversation_history = []

# Función para generar respuestas
def generate_response(message, history):
    # Construir el contexto del chat con formato para Llama-2
    prompt = ""
    for user_msg, bot_msg in history:
        prompt += f"<s>[INST] {user_msg} [/INST] {bot_msg}</s>"

    # Añadir el mensaje actual
    prompt += f"<s>[INST] {message} [/INST]"

    # Generar respuesta
    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)
    outputs = model.generate(
        inputs.input_ids,
        max_new_tokens=512,
        do_sample=True,
        temperature=0.7,
        top_p=0.9,
    )

    # Decodificar la respuesta
    response = tokenizer.decode(outputs[0][inputs.input_ids.shape[1]:], skip_special_tokens=True)

    return response

# Crear la interfaz de Gradio
demo = gr.ChatInterface(
    fn=generate_response,
    title="Chat con Llama-2-7b",
    description="Un chatbot simple usando el modelo Llama-2-7b. Haz una pregunta para empezar.",
    examples=["¿Qué es un transformer en IA?", "Explica el concepto de atención en NLP", "¿Cómo funciona GPT?"],
    theme="soft"
)

# Lanzar la app
demo.launch(share=True)  # share=True crea un enlace público